In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region= "donor"
cancer_type= "Brain"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_acceptor = pd.read_csv("/home/pdutta/Data/Noncoding_region/Unique_{}_RAW_Combined_BED.tsv".format(non_coding_region), sep="\t")
df_acceptor

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12189,12268,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001671638.2, ENSE00002234944.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12659,12738,+,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene
2,chr1,12683,12762,+,ENST00000456328.2,ENSE00003582793.1,processed_transcript
3,chr1,13336,13415,+,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene
4,chr1,15756,15835,-,ENST00000488147.1,ENSE00002030414.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
297951,chrY,57211531,57211610,+,ENST00000464205.6_PAR_Y,ENSE00003785071.1,processed_transcript
297952,chrY,57211582,57211661,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00003627301.1, ENSE00003905275.1, ENSE0000...","retained_intron, protein_coding"
297953,chrY,57213164,57213243,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
297954,chrY,57213840,57213919,-,ENST00000507418.6_PAR_Y,ENSE00002046926.1,unprocessed_pseudogene


In [6]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [7]:
len(loaded_dictionary)

432

In [8]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [9]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'START_POS', 'END_POS', 'REF','ALT']).reset_index()
        concat_probab = np.load(dir_path+"/Prediction_result/pred_results.npy")
        print(len(concat_probab))
        print(len(df_temp))
        if(2*len(df_temp)==len(concat_probab)):
            #input()
            Reference_probab = concat_probab[::2]
            Alternative_probab = concat_probab[1::2]
            df_temp['Ref_probab']=Reference_probab
            df_temp['Alt_probab']=Alternative_probab
            df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
            df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
            #display(df_temp)
            #input()
            # Apply the filter conditions
            df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                      ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
            #display(df_temp)
            #input()
            patient_dfs.append(df_temp)
        else:
            print(dir_name)
            input()
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/donor/Patient_wise ['a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel', '0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel', '3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan', 'af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel', '67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan', '447a08c8-3488-45d1-9262-23c42341296f_sanger_raw_pindel', '033db366-1af1-4bd9-8339-c54800b1de57_sanger_raw_pindel', 'f15d872a-7e31-4111-9d92-5ac813a8bc4f_sanger_raw_pindel', 'c6b9e176-a36b-4eb2-aa15-6a0155c1aeb5_CaVEMan', 'ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_raw_pindel', '3b89e48e-3e35-402f-b7f0-6e773839de4e_sanger_raw_pindel', '4132ab70-457f-41a9-aae3-340f9fe109e2_sanger_raw_pindel', 'd214490c-9f1c-4228-8cbc-7c8238942041_CaVEMan', '4e285b95-27ba-49b9-980a-6dbfc09a62c0_CaVEMan', '410db725-9a2f-4df3-8ace-6eedd4988cec_CaVEMan', 'b845bff2-f0eb-4375-95f0-0ebecab03c29_CaVEMan', '535e0e9e-9757-45b3-aafd-2b80908de134_CaVEMan

In [11]:
patient_dfs[0]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
6,7,chr1,1387737,1387816,-,"ENST00000400809.8, ENST00000408952.8, ENST0000...","ENSE00003503679.1, ENSE00003635706.1","protein_coding, retained_intron, nonsense_medi...",chr1,1387762,...,D,1387766,1,1387763,374,3134.949951171875,0.169704,0.626758,0.286463,-3.038407
12,17,chr1,3836455,3836534,-,"ENST00000378230.8, ENST00000428079.6, ENST0000...","ENSE00003569347.1, ENSE00003900508.1, ENSE0000...","protein_coding, retained_intron, nonsense_medi...",chr1,3836467,...,I,3836493,23,3836468,12,414.00799560546875,0.008272,0.816154,0.659356,-9.055833
23,31,chr1,15524029,15524108,-,"ENST00000333868.10, ENST00000348549.9, ENST000...","ENSE00002686918.2, ENSE00002428689.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,15524057,...,I,15524066,0,15524058,84,1443.22998046875,0.846076,0.493081,-0.298661,2.498506
26,34,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052445,...,DI,16052451,0,16052445,21,.,0.575705,0.382741,-0.111090,1.129770
49,72,chr1,35079404,35079483,+,"ENST00000359858.9, ENST00000373329.5, ENST0000...","ENSE00001460290.4, ENSE00001374249.1, ENSE0000...","protein_coding, processed_transcript",chr1,35079420,...,I,35079423,0,35079421,391,3173.320068359375,0.069182,0.990067,0.911738,-10.389182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,3448,chr9,137162050,137162129,+,"ENST00000350902.9, ENST00000371546.8, ENST0000...","ENSE00003601294.1, ENSE00003648634.1","nonsense_mediated_decay, protein_coding, retai...",chr9,137162099,...,D,137162112,1,137162100,72,1356.3299560546875,0.554890,0.407306,-0.081893,0.859211
2609,3473,chrX,47659363,47659442,+,ENST00000670339.1,ENSE00003858131.1,lncRNA,chrX,47659399,...,D,47659403,2,47659400,63,1194.72998046875,0.767078,0.340972,-0.326856,2.670210
2612,3476,chrX,49174387,49174466,+,ENST00000376327.6,ENSE00000670265.1,protein_coding,chrX,49174428,...,I,49174444,6,49174429,84,1430.27001953125,0.466589,0.642324,0.112879,-1.037746
2637,3504,chrX,151694705,151694784,+,"ENST00000652529.1, ENST00000674441.1","ENSE00003844954.1, ENSE00003898736.1",protein_coding,chrX,151694750,...,D,151694758,6,151694751,110,1617.2099609375,0.520489,0.470029,-0.026264,0.291465


In [12]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])
unique_patient_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
6,7,chr1,1387737,1387816,-,"ENST00000400809.8, ENST00000408952.8, ENST0000...","ENSE00003503679.1, ENSE00003635706.1","protein_coding, retained_intron, nonsense_medi...",chr1,1387762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,17,chr1,3836455,3836534,-,"ENST00000378230.8, ENST00000428079.6, ENST0000...","ENSE00003569347.1, ENSE00003900508.1, ENSE0000...","protein_coding, retained_intron, nonsense_medi...",chr1,3836467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,31,chr1,15524029,15524108,-,"ENST00000333868.10, ENST00000348549.9, ENST000...","ENSE00002686918.2, ENSE00002428689.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,15524057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,34,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,72,chr1,35079404,35079483,+,"ENST00000359858.9, ENST00000373329.5, ENST0000...","ENSE00001460290.4, ENSE00001374249.1, ENSE0000...","protein_coding, processed_transcript",chr1,35079420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,131,chr11,64653656,64653735,-,"ENST00000265459.11, ENST00000409571.5, ENST000...","ENSE00003546612.1, ENSE00003460886.1","protein_coding, processed_transcript",chr11,64653695,...,0.0,128.0,.,1.000000e-10,0.83,TT/TTTTT,False,0.170,TT/GTTTT,0.83
312,312,chr16,55733472,55733551,+,ENST00000566467.2,ENSE00002583592.1,unprocessed_pseudogene,chr16,55733540,...,0.0,55.0,.,3.900000e-03,0.97,AA/AAAAA,False,0.021,AA/AAAAT,0.97
408,408,chr18,46483547,46483626,-,"ENST00000300591.11, ENST00000398686.8, ENST000...",ENSE00001111001.1,protein_coding,chr18,46483557,...,0.0,125.0,.,2.800000e-08,1.00,AA/AAGGG,False,0.098,AA/AAAGG,0.90
730,730,chr3,49422321,49422400,-,"ENST00000273588.9, ENST00000395338.7, ENST0000...","ENSE00003794373.1, ENSE00002029851.3, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr3,49422337,...,0.0,92.0,.,3.100000e-06,0.99,AA/AAAAA,False,0.010,AA/AAAAC,0.99


In [13]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
26,34,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,127,chr1,100729344,100729423,+,"ENST00000294728.7, ENST00000370119.8, ENST0000...","ENSE00001067756.1, ENSE00003836946.1","protein_coding, nonsense_mediated_decay",chr1,100729408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,138,chr1,118081061,118081140,-,"ENST00000336338.10, ENST00000473472.1, ENST000...","ENSE00003569049.1, ENSE00001861494.1, ENSE0000...","protein_coding, processed_transcript",chr1,118081057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,315,chr1,225863902,225863981,-,ENST00000474478.5,ENSE00001820008.2,processed_transcript,chr1,225863937,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,353,chr10,4497437,4497516,+,ENST00000660243.1,ENSE00003877621.1,lncRNA,chr10,4497450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,159,chr1,118081061,118081140,-,"ENST00000336338.10, ENST00000473472.1, ENST000...","ENSE00003569049.1, ENSE00001861494.1, ENSE0000...","protein_coding, processed_transcript",chr1,118081057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,1105,chr15,84638439,84638518,+,"ENST00000348993.9, ENST00000558508.1, ENST0000...","ENSE00001238362.1, ENSE00002563829.1",processed_transcript,chr15,84638487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2448,3011,chr7,106775984,106776063,+,"ENST00000470135.5, ENST00000655590.1",ENSE00001950886.1,lncRNA,chr7,106776006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2580,3172,chr8,89784101,89784180,+,"ENST00000220751.5, ENST00000522965.1","ENSE00003617726.1, ENSE00003539198.1","protein_coding, nonsense_mediated_decay",chr8,89784140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==16052373]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
25,30,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052445,16052450,98f8a55e-9a6b-11ea-8b18-a1f006b0d7b9,GTCAC,TCAT,240.0,FF010,True,5.0,"('chr1:16052445:GGTCAC/GTCAT',)",DI,16052451.0,0.0,16052445.0,9.0,.,0.576383,0.383728,-0.111043,1.127752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,51,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052440,16052442,eb96b394-47fb-11eb-a5ac-902aa0570f16,AG,A,2454.0,FF010,True,1.0,.,D,16052447.0,5.0,16052441.0,480.0,3527.570068359375,0.576383,0.426812,-0.086210,0.869677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
unique_patient_regions[unique_patient_regions['start']==1043198]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
0,0,chr1,1043198,1043277,+,"ENST00000379370.7, ENST00000620552.4, ENST0000...",ENSE00001605257.1,protein_coding,chr1,1043222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(191, 29)
(187, 29)
(105, 32)
(189, 29)
(10, 32)
(182, 29)
(173, 29)
(180, 29)
(12, 32)
(246, 29)
(192, 29)
(193, 29)
(7, 32)
(9, 32)
(93, 32)
(6, 32)
(6, 32)
(6, 32)
(196, 29)
(186, 29)
(5, 32)
(208, 29)
(183, 29)
(5, 32)
(10, 32)
(175, 29)
(10, 32)
(3, 32)
(253, 29)
(49, 32)
(7, 32)
(174, 29)
(7, 32)
(178, 29)
(185, 29)
(191, 29)
(181, 29)
(5, 32)
(196, 29)
(4, 32)
(9, 32)
(185, 29)
(188, 29)
(3, 32)
(9, 32)
(5, 32)
(182, 29)
(71, 32)
(5, 32)
(5, 32)
(183, 29)
(14, 32)
(197, 29)
(202, 29)
(5, 32)
(173, 29)
(181, 29)
(70, 32)
(5, 32)
(13, 32)
(62, 32)
(191, 29)
(6, 32)
(7, 32)
(182, 29)
(6, 32)
(5, 32)
(178, 29)
(168, 29)
(3, 32)
(190, 29)
(3, 32)
(177, 29)
(181, 29)
(182, 29)
(175, 29)
(181, 29)
(180, 29)
(6, 32)
(9, 32)
(6, 32)
(216, 29)
(14, 32)
(182, 29)
(6, 32)
(6, 32)
(8, 32)
(173, 29)
(194, 29)
(109, 32)
(177, 29)
(181, 29)
(6, 32)
(176, 29)
(183, 29)
(5, 32)
(183, 29)
(176, 29)
(2, 32)
(205, 29)
(185, 29)
(189, 29)
(192, 29)
(185, 29)
(5, 32)
(53, 32)
(5, 32)
(12, 32)
(66, 32)

In [15]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [16]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

,Patient_ID,chr1_1387737_1387816_1387762_1387765_CCT_C,chr1_3836455_3836534_3836467_3836468_G_GT,chr1_15524029_15524108_15524057_15524058_C_CGGGGGCG,chr1_16052373_16052452_16052445_16052450_GTCAC_TCAT,chr1_35079404_35079483_35079420_35079421_G_GGC,chr1_55056151_55056230_55056225_55056233_GGGGCGGA_G,chr1_59762631_59762710_59762637_59762638_T_TTC,chr1_63014040_63014119_63014041_63014042_G_GCTA,chr1_93846925_93847004_93846925_93846926_T_TC,...,chr11_104606983_104607062_104607015_104607021_GCCTAA_G,chr19_32040532_32040611_32040534_32040535_A_AC,chr22_20983084_20983163_20983088_20983093_GGTGA_G,chr22_23583769_23583848_23583797_23583803_GCTCCT_G,chrX_120143862_120143941_120143876_120143877_G_GA,chr11_64653656_64653735_64653695_64653696_T_G,chr16_55733472_55733551_55733540_55733541_A_T,chr18_46483547_46483626_46483557_46483558_A_G,chr3_49422321_49422400_49422337_49422338_A_C,chr4_70681561_70681640_70681600_70681601_T_C
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.169704,0.008272,0.846076,0.575705,0.069182,0.460225,0.005666,0.018851,0.081147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.169704,0.008272,0.846076,0.575705,0.069182,0.460225,0.005666,NaN,0.081147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,0.169704,0.008272,0.846076,NaN,0.069182,0.460225,0.005666,0.018851,0.081147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,0.169704,NaN,NaN,NaN,0.069182,0.460225,0.005666,0.018851,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,0.008272,0.846076,NaN,0.069182,0.460225,0.005666,NaN,0.081147,...,0.579511,0.30353,0.01226,0.073325,0.125107,NaN,NaN,NaN,NaN,NaN


In [17]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

,Patient_ID,chr1_1387737_1387816_1387762_1387765_CCT_C,chr1_3836455_3836534_3836467_3836468_G_GT,chr1_15524029_15524108_15524057_15524058_C_CGGGGGCG,chr1_16052373_16052452_16052445_16052450_GTCAC_TCAT,chr1_35079404_35079483_35079420_35079421_G_GGC,chr1_55056151_55056230_55056225_55056233_GGGGCGGA_G,chr1_59762631_59762710_59762637_59762638_T_TTC,chr1_63014040_63014119_63014041_63014042_G_GCTA,chr1_93846925_93847004_93846925_93846926_T_TC,...,chr11_104606983_104607062_104607015_104607021_GCCTAA_G,chr19_32040532_32040611_32040534_32040535_A_AC,chr22_20983084_20983163_20983088_20983093_GGTGA_G,chr22_23583769_23583848_23583797_23583803_GCTCCT_G,chrX_120143862_120143941_120143876_120143877_G_GA,chr11_64653656_64653735_64653695_64653696_T_G,chr16_55733472_55733551_55733540_55733541_A_T,chr18_46483547_46483626_46483557_46483558_A_G,chr3_49422321_49422400_49422337_49422338_A_C,chr4_70681561_70681640_70681600_70681601_T_C
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.626758,0.816154,0.493081,0.382741,0.990067,0.565128,0.992409,0.957858,0.966388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.626758,0.816154,0.493081,0.382741,0.990067,0.565128,0.992409,NaN,0.966388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,0.626758,0.816154,0.493081,NaN,0.990067,0.565128,0.992409,0.957858,0.966388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,0.626758,NaN,NaN,NaN,0.990067,0.565128,0.992409,0.957858,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,0.816154,0.493081,NaN,0.990067,0.565128,0.992409,NaN,0.966388,...,0.030291,0.946446,0.937489,0.923203,0.994429,NaN,NaN,NaN,NaN,NaN


In [18]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

,Patient_ID,chr1_1387737_1387816_1387762_1387765_CCT_C,chr1_3836455_3836534_3836467_3836468_G_GT,chr1_15524029_15524108_15524057_15524058_C_CGGGGGCG,chr1_16052373_16052452_16052445_16052450_GTCAC_TCAT,chr1_35079404_35079483_35079420_35079421_G_GGC,chr1_55056151_55056230_55056225_55056233_GGGGCGGA_G,chr1_59762631_59762710_59762637_59762638_T_TTC,chr1_63014040_63014119_63014041_63014042_G_GCTA,chr1_93846925_93847004_93846925_93846926_T_TC,...,chr11_104606983_104607062_104607015_104607021_GCCTAA_G,chr19_32040532_32040611_32040534_32040535_A_AC,chr22_20983084_20983163_20983088_20983093_GGTGA_G,chr22_23583769_23583848_23583797_23583803_GCTCCT_G,chrX_120143862_120143941_120143876_120143877_G_GA,chr11_64653656_64653735_64653695_64653696_T_G,chr16_55733472_55733551_55733540_55733541_A_T,chr18_46483547_46483626_46483557_46483558_A_G,chr3_49422321_49422400_49422337_49422338_A_C,chr4_70681561_70681640_70681600_70681601_T_C
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.286463,0.659356,-0.298661,-0.11109,0.911738,0.059284,0.979252,0.899435,0.855487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.286463,0.659356,-0.298661,-0.11109,0.911738,0.059284,0.979252,NaN,0.855487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,0.286463,0.659356,-0.298661,NaN,0.911738,0.059284,0.979252,0.899435,0.855487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,0.286463,NaN,NaN,NaN,0.911738,0.059284,0.979252,0.899435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,0.659356,-0.298661,NaN,0.911738,0.059284,0.979252,NaN,0.855487,...,-0.318279,0.608486,0.867392,0.78461,0.864479,NaN,NaN,NaN,NaN,NaN


In [19]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

,Patient_ID,chr1_1387737_1387816_1387762_1387765_CCT_C,chr1_3836455_3836534_3836467_3836468_G_GT,chr1_15524029_15524108_15524057_15524058_C_CGGGGGCG,chr1_16052373_16052452_16052445_16052450_GTCAC_TCAT,chr1_35079404_35079483_35079420_35079421_G_GGC,chr1_55056151_55056230_55056225_55056233_GGGGCGGA_G,chr1_59762631_59762710_59762637_59762638_T_TTC,chr1_63014040_63014119_63014041_63014042_G_GCTA,chr1_93846925_93847004_93846925_93846926_T_TC,...,chr11_104606983_104607062_104607015_104607021_GCCTAA_G,chr19_32040532_32040611_32040534_32040535_A_AC,chr22_20983084_20983163_20983088_20983093_GGTGA_G,chr22_23583769_23583848_23583797_23583803_GCTCCT_G,chrX_120143862_120143941_120143876_120143877_G_GA,chr11_64653656_64653735_64653695_64653696_T_G,chr16_55733472_55733551_55733540_55733541_A_T,chr18_46483547_46483626_46483557_46483558_A_G,chr3_49422321_49422400_49422337_49422338_A_C,chr4_70681561_70681640_70681600_70681601_T_C
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,-3.038407,-9.055833,2.498506,1.12977,-10.389182,-0.608007,-14.485707,-10.208243,-8.346789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,-3.038407,-9.055833,2.498506,1.12977,-10.389182,-0.608007,-14.485707,NaN,-8.346789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,-3.038407,-9.055833,2.498506,NaN,-10.389182,-0.608007,-14.485707,-10.208243,-8.346789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,-3.038407,NaN,NaN,NaN,-10.389182,-0.608007,-14.485707,-10.208243,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,-9.055833,2.498506,NaN,-10.389182,-0.608007,-14.485707,NaN,-8.346789,...,5.463381,-5.341689,-10.238668,-7.247218,-10.2857,NaN,NaN,NaN,NaN,NaN


In [20]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)